# Prerequisites

1. Create a project on Google Developers console
2. Enable Google drive api for the project
3. Setup credentials and OAuth, download the credentials and save them as client_secrets.json in working directory. This <a href='https://codelabs.developers.google.com/codelabs/gsuite-apis-intro/#5'>guide</a> can be useful for setting up credentials and OAuth. And this <a href='https://pythonhosted.org/PyDrive/quickstart.html#authentication'>guide</a> for introduction to pydrive


In [6]:
import os
import shutil
import random
import glob
import re
from zipfile import ZipFile

# Uploading the files

In [1]:
from pydrive.auth import GoogleAuth
#os.chdir('/path/to/working/directory')
gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=709211786339-8me0rsaice79ih2jifhtcqgl8536oakq.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [2]:
from pydrive.drive import GoogleDrive

drive = GoogleDrive(gauth)

## Listing directories inside a shared drive or inside a folder

In [3]:
team_drive_id = "<team_drive_id>"#this id is in the url after the last slash

query = "'"+str(team_drive_id)+"' in parents and trashed=False"
#you can put folder_id in query as well

file_list = drive.ListFile({'q':query,
                            'corpora':'teamDrive',
                            'teamDriveId':team_drive_id,
                            'includeTeamDriveItems': True, 
                            'supportsTeamDrives' : True,
                            }).GetList()
for f in file_list:
    print(f['title'], f['id'])

ADNIMERGE.csv 1npaNJIjQLB1hYQcn4YrT_113D-zRF639
Nabarun 1vrckPbtoIPNuT4KGzD39DEFkT-wLOVQ2
ADNI Labeled 1viLS9qQhHRIu6vJEvoIf6MIqlo45Y0iB
ADNI Longitudinal 1aM8sZRpnCP4EwUA1UkqCP4q5fElqPylj


## Creating a test file inside a folder on the shared drive
This worked perfectly in the afternoon but is giving shared drive file llimit reached now

In [4]:
team_drive_id = "<team_drive_id>"
MCI_NC_folder_id = "<folder_id>"

f = drive.CreateFile({
    'title': 'test_api.txt',
    'parents': [{
        'kind': 'drive#fileLink',
        'teamDriveId': team_drive_id,
        'id': MCI_NC_folder_id
    }]
})
f.SetContentString('Testing the google drive api upload.')

f.Upload(param={'supportsTeamDrives': True})

ApiRequestError: <HttpError 403 when requesting https://www.googleapis.com/upload/drive/v2/files?supportsTeamDrives=true&alt=json&uploadType=resumable returned "The file limit for this shared drive has been exceeded.">

## Navigating a directory structure on own system

In [7]:
num_files = 0

def list_all_files(path):
    global num_files
    num_files+=1
    
    if os.path.isfile(path):
        #print(path)
        pass
    elif os.path.isdir(path):
        dir_name = re.search('ADNI/(.+?)$',path).group(1)
        print("Entering directory {}".format(dir_name))
        os.chdir(path)
        all_p = os.listdir('.')
        #print(all_p)
        for p in all_p:
            list_all_files(os.path.join(path,p))
    
    pass


src_directory = '/path/to/subject/folder'

all_paths = glob.glob(os.path.join(src_directory,'*'))

for path in all_paths:
    list_all_files(path)
print(num_files)

Entering directory 002_S_4746/Resting_State_fMRI
Entering directory 002_S_4746/Resting_State_fMRI/2013-06-03_15_00_48.0
Entering directory 002_S_4746/Resting_State_fMRI/2013-06-03_15_00_48.0/S191322
Entering directory 002_S_4746/Resting_State_fMRI/2012-12-11_15_21_53.0
Entering directory 002_S_4746/Resting_State_fMRI/2012-12-11_15_21_53.0/S178221
Entering directory 002_S_4746/Resting_State_fMRI/2012-09-11_13_44_50.0
Entering directory 002_S_4746/Resting_State_fMRI/2012-09-11_13_44_50.0/S167243
Entering directory 002_S_4746/Resting_State_fMRI/2012-05-29_16_35_01.0
Entering directory 002_S_4746/Resting_State_fMRI/2012-05-29_16_35_01.0/S151916
Entering directory 002_S_4746/MPRAGE
Entering directory 002_S_4746/MPRAGE/2013-06-03_15_00_48.0
Entering directory 002_S_4746/MPRAGE/2013-06-03_15_00_48.0/S191323
Entering directory 002_S_4746/MPRAGE/2012-05-29_16_37_42.0
Entering directory 002_S_4746/MPRAGE/2012-05-29_16_37_42.0/S151914
Entering directory 002_S_4746/MPRAGE/2012-12-11_15_29_16.0
Ent

## Uploading nested directory structure on shared drive

In [8]:
def create_folder(drive, folder_name, parent_folder_id, team_drive_id):
    
    folder_metadata = {
        'title': folder_name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [{
            'kind': 'drive#fileLink',
            'teamDriveId': team_drive_id,
            'id': parent_folder_id
        }]
    }

    folder = drive.CreateFile(folder_metadata)
    folder.Upload(param={'supportsTeamDrives': True})

    # Return folder informations
    print('Created folder title: {}, id: {}'.format(folder['title'], folder['id']))
    return (folder['id'])


def upload_file(drive, file_path, destination_dir_id, team_drive_id):
    title = file_path.split('/')[-1]
    f = drive.CreateFile({
        'title':title,
        "parents": [{
            "kind": "drive#fileLink", 
            'teamDriveId': team_drive_id, 
            "id": destination_dir_id
        }]}
    )
    f.SetContentFile(file_path)
    f.Upload(param={'supportsTeamDrives': True})
    pass

In [9]:
def upload_directory(drive, src_dir_path, destination_dir_id, team_drive_id):
    
    folder_name = src_dir_path.split('/')[-1]
    new_folder_id = create_folder(drive, folder_name, destination_dir_id, team_drive_id)
    os.chdir(src_dir_path)
    all_paths = os.listdir('.')
    
    for p in all_paths:
        full_path = os.path.join(src_dir_path,p)
        if os.path.isfile(full_path):
            upload_file(drive, full_path, new_folder_id, team_drive_id)
        elif os.path.isdir(full_path):
            upload_directory(drive, full_path, new_folder_id, team_drive_id)
        else:
            print("Path {} is neither diretory nor fle".format(p))
            
    pass

# The drive id and folder id are the last part of the url
I was not able to make sure whether this part worked correctly since I was getting file limit reached from the first trail itself.

In [11]:
team_drive_id = "<team_drive_id>" #last part of url for the shared drive
MCI_NC_folder_id = "<folder_id>"  #last part of url for the folder

src_dir_path = "/path/to/ADNI/directory/or/test/directory"

upload_directory(drive, src_dir_path, MCI_NC_folder_id, team_drive_id)

ApiRequestError: <HttpError 403 when requesting https://www.googleapis.com/drive/v2/files?supportsTeamDrives=true&alt=json returned "The file limit for this shared drive has been exceeded.">

## Uploading nested directory structure on own drive
This works as expected but is slow

In [12]:
def create_folder(drive, folder_name, parent_folder_id):
    
    folder_metadata = {
        'title': folder_name,
        'mimeType': 'application/vnd.google-apps.folder',
        'parents': [{
            'kind': 'drive#fileLink',
            #'teamDriveId': team_drive_id,
            'id': parent_folder_id
        }]
    }

    folder = drive.CreateFile(folder_metadata)
    #folder.Upload(param={'supportsTeamDrives': True})
    folder.Upload()
    
    # Return folder informations
    print('Created folder title: {}, id: {}'.format(folder['title'], folder['id']))
    return (folder['id'])


def upload_file(drive, file_path, destination_dir_id):
    title = file_path.split('/')[-1]
    f = drive.CreateFile({
        'title':title,
        "parents": [{
            "kind": "drive#fileLink", 
            #'teamDriveId': team_drive_id, 
            "id": destination_dir_id
        }]}
    )
    f.SetContentFile(file_path)
    #f.Upload(param={'supportsTeamDrives': True})
    f.Upload()
    pass

In [13]:
def upload_directory(drive, src_dir_path, destination_dir_id):
    
    folder_name = src_dir_path.split('/')[-1]
    new_folder_id = create_folder(drive, folder_name, destination_dir_id)
    os.chdir(src_dir_path)
    all_paths = os.listdir('.')
    
    for p in all_paths:
        full_path = os.path.join(src_dir_path,p)
        if os.path.isfile(full_path):
            upload_file(drive, full_path, new_folder_id)
        elif os.path.isdir(full_path):
            upload_directory(drive, full_path, new_folder_id)
        else:
            print("Path {} is neither diretory nor fle".format(p))
            
    pass

In [14]:
#team_drive_id = ">team_drive_id>"
folder_id = "<folder_id>"

src_dir_path = "/path/to/ADNI/or/test/folder"

upload_directory(drive, src_dir_path, folder_id)

Created folder title: test_api1, id: 1xI84aKKxWVX82lYImagKqfDFoAQ23Nj6
Created folder title: sid2, id: 1ew6GW7u9ZqmUhUiGMYxuUtPryOtZnJof
Created folder title: m1, id: 1E3_VFIwU-QnFqH1rzgveM-f-yriep6EL
Created folder title: date3, id: 108so0EOkt7g7Zvg7xRBw7LHjmoYLyIsZ
Created folder title: date1, id: 18vF8H4GgZadMg2Y7MYeCGXLGgHX2ysFo
Created folder title: date2, id: 1Mtlp3LDnKipdP3cwvkaUnO2JpNpmobau
Created folder title: m2, id: 1GxFpnM2ICTSEELPAetrv2IQA56EfvF3f
Created folder title: date3, id: 1u6WA4OCrPSQOZV2bc3Ma6mxnv82GcSbe
Created folder title: date1, id: 1cZS1GGtjy171CU4BNz3Sq-XhoAUjqFU6
Created folder title: date2, id: 1RYxwO4Z2IAqFzQxidgMuUYDlsm4FDZvP
Created folder title: m3, id: 1dp7CBZX9t2catnbyPUbG6zDFEwv35ZGn
Created folder title: date3, id: 1LhoDzEid9OWlmzbOwYcBq6Ly-ShGc20j
Created folder title: date1, id: 1BC20L0YhzH0FT7Qkh2Nr6Aigz7OYYGsM
Created folder title: date2, id: 1LU2g2M5Wr_lBIiZh-TGNPpOPSeOUeyIe
Created folder title: sid1, id: 1neEbS5ao67SCtm51i1HkMK0eF03lBWvW
Cr